In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf

In [2]:
data = pd.read_csv(r'C:\Users\zig53\vscode jupyther\Datasets\BTC-USD_6years.csv')
data.head()


,datetime,symbol,open,high,low,close,volume
0,2016-06-05 07:00:00,BITFINEX:BTCUSD,573.08,586.47,564.64,574.02,28143.301081
1,2016-06-06 07:00:00,BITFINEX:BTCUSD,574.01,588.60,573.93,585.34,27464.294823
2,2016-06-07 07:00:00,BITFINEX:BTCUSD,585.59,592.08,561.30,576.88,36112.753034
3,2016-06-08 07:00:00,BITFINEX:BTCUSD,577.23,585.15,570.62,583.05,18084.176162
4,2016-06-09 07:00:00,BITFINEX:BTCUSD,583.26,584.05,569.97,575.52,21873.892421


In [3]:
df = data.copy()
df.ta.macd(append = True)
df.columns

Index(['datetime', 'symbol', 'open', 'high', 'low', 'close', 'volume',
       'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9'],
      dtype='object')

In [4]:
df['signal'] = df.iloc[:, -3] > df.iloc[:,-1] # macd > signal
signal_vectorbt = df.ta.tsignals(df.signal, asbool = True, append = True) #as boolean give a signal to True or False

In [5]:
df.head()

,datetime,symbol,open,high,low,close,volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,signal,TS_Trends,TS_Trades,TS_Entries,TS_Exits
0,2016-06-05 07:00:00,BITFINEX:BTCUSD,573.08,586.47,564.64,574.02,28143.301081,NaN,NaN,NaN,False,False,0,False,False
1,2016-06-06 07:00:00,BITFINEX:BTCUSD,574.01,588.60,573.93,585.34,27464.294823,NaN,NaN,NaN,False,False,0,False,False
2,2016-06-07 07:00:00,BITFINEX:BTCUSD,585.59,592.08,561.30,576.88,36112.753034,NaN,NaN,NaN,False,False,0,False,False
3,2016-06-08 07:00:00,BITFINEX:BTCUSD,577.23,585.15,570.62,583.05,18084.176162,NaN,NaN,NaN,False,False,0,False,False
4,2016-06-09 07:00:00,BITFINEX:BTCUSD,583.26,584.05,569.97,575.52,21873.892421,NaN,NaN,NaN,False,False,0,False,False


In [6]:
# make buy order after candlestick price is closed
trades_table = df.iloc[:,-5:][df.TS_Trades != 0] 
trades_table

,signal,TS_Trends,TS_Trades,TS_Entries,TS_Exits
42,True,True,1,True,False
47,False,False,-1,False,True
70,True,True,1,True,False
97,False,False,-1,False,True
111,True,True,1,True,False
...,...,...,...,...,...
2103,True,True,1,True,False
2123,False,False,-1,False,True
2152,True,True,1,True,False
2153,False,False,-1,False,True


In [7]:
trades_table['action_price'] = df['open'].shift(-1)
trades_table['return'] = trades_table['action_price'].pct_change()
trades_table.head()

,signal,TS_Trends,TS_Trades,TS_Entries,TS_Exits,action_price,return
42,True,True,1,True,False,683.18,NaN
47,False,False,-1,False,True,648.11,-0.051333
70,True,True,1,True,False,581.50,-0.102776
97,False,False,-1,False,True,607.98,0.045537
111,True,True,1,True,False,614.08,0.010033


In [11]:
## use return after sell or Exits is True
trades_table.reset_index(drop = True, inplace = True)

trades_summary = trades_table.loc[trades_table['TS_Exits'] == True]

trades_summary.head()


,signal,TS_Trends,TS_Trades,TS_Entries,TS_Exits,action_price,return
1,False,False,-1,False,True,648.11,-0.051333
3,False,False,-1,False,True,607.98,0.045537
5,False,False,-1,False,True,632.46,0.029931
7,False,False,-1,False,True,705.42,0.060798
9,False,False,-1,False,True,739.00,-0.019777


In [17]:
compare = []
trade_log = []
for i in range(11,15):
      for j in range(21, 26):

        #reset dataframe
        df = data.copy()

        #run macd
        df.ta.macd(fast=i,slow=j,append=True)

        #create signal
        df['signal'] = df.iloc[:,-3] > df.iloc[:,-1]

        #create TS table
        signal_vectorbt = df.ta.tsignals(df.signal, asbool=True, append=True)

        #action on next open 
        df['action_price'] = df['open'].shift(-1)

        #create trade log
        trades_table = df.iloc[:,-5:][df.TS_Trades !=0]
        trades_table['return'] = trades_table['action_price'].pct_change()

        trades_summary = trades_table.loc[trades_table.TS_Exits==True]

        #select data for compare
        a = trades_summary['return']
        trade_log.append(a)
        
        b = [a.mean(),a.median(),a.min(),(i,j)]
        compare.append(b)

In [19]:
trades_summary

,TS_Trends,TS_Trades,TS_Entries,TS_Exits,action_price,return
34,False,-1,False,True,648.120000,-0.043153
47,False,-1,False,True,648.110000,-0.025882
99,False,-1,False,True,609.790000,0.037040
131,False,-1,False,True,635.910000,0.035549
148,False,-1,False,True,705.420000,0.060798
...,...,...,...,...,...,...
2049,False,-1,False,True,36463.160761,-0.153791
2077,False,-1,False,True,40027.000000,0.047783
2093,False,-1,False,True,38437.048386,-0.110418
2124,False,-1,False,True,43194.821329,0.049973


In [20]:
result = pd.DataFrame(compare,columns = ['Avg Return','Median','Worst Trade Return','MACD Parameter Set']).set_index('MACD Parameter Set',drop=True)
result

,Avg Return,Median,Worst Trade Return
MACD Parameter Set,,,
"(11, 21)",0.062021,-0.010495,-0.149052
"(11, 22)",0.063847,-0.010482,-0.149052
"(11, 23)",0.064820,-0.009977,-0.153791
"(11, 24)",0.062727,-0.009679,-0.153791
"(12, 21)",0.064727,-0.010482,-0.153791
"(12, 22)",0.063737,-0.009931,-0.153791
"(12, 23)",0.071312,-0.008299,-0.153791
"(12, 24)",0.075695,-0.005874,-0.170826
"(13, 21)",0.070539,-0.019619,-0.153791
